In [1]:
!pip install kaggle
!pip install speechrecognition
!pip install transformers
!pip install torch
!pip install soundfile
!pip install librosa
!pip install gradio  

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
    --------------------------------------- 0.3/18.1 MB ? eta -:--:--
    --------------------------------------- 0.3/18.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.1 MB 578.7 kB/s eta 0:00:31
   - -------------------------------------- 0.5/18.1 MB 578.7 kB/s eta 0:00:31
   - -------------------------------------- 0.5/18.1 MB 578.7 kB/s eta 0:00:31
   - -------------------------------------- 0.8/18.1 MB 516.0 kB/s eta 0:00:34
   - -------------------------------------- 0.8/18.1 MB 516.0 kB/s eta 0:00:34
   -- ------------------------------------- 1.0/18.1 MB 503.

In [6]:
!kaggle datasets list -s grammar

ref                                                       title                                                   size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -----------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
datafiniti/grammar-and-online-product-reviews             Grammar and Online Product Reviews                   9383592  2018-02-15 17:20:27                  5175        104  0.7058824        
dariocioni/c4200m                                         C4 200M Grammar Error Correction dataset         15601869562  2023-04-18 16:46:11                  1909         44  1.0              
satishgunjal/grammar-correction                           Grammar Correction                                     63861  2023-12-19 06:39:41.163000            780         10  0.9411765        
harshildarji/reber                      

In [7]:
!kaggle datasets download -d satishgunjal/grammar-correction

Dataset URL: https://www.kaggle.com/datasets/satishgunjal/grammar-correction
License(s): apache-2.0


In [8]:
import zipfile

with zipfile.ZipFile("grammar-correction.zip", 'r') as zip_ref:
    zip_ref.extractall("grammar_dataset") 

In [21]:
import pandas as pd

df = pd.read_csv("grammar_dataset/Grammar_Correction.csv")
df.head()


,Serial Number,Error Type,Ungrammatical Statement,Standard English
0,1,Verb Tense Errors,I goes to the store everyday.,I go to the store everyday.
1,2,Verb Tense Errors,They was playing soccer last night.,They were playing soccer last night.
2,3,Verb Tense Errors,She have completed her homework.,She has completed her homework.
3,4,Verb Tense Errors,He don't know the answer.,He doesn't know the answer.
4,5,Verb Tense Errors,The sun rise in the east.,The sun rises in the east.


In [22]:
df.info()
df.sample(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018 entries, 0 to 2017
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Serial Number            2018 non-null   int64 
 1   Error Type               2018 non-null   object
 2   Ungrammatical Statement  2018 non-null   object
 3   Standard English         2018 non-null   object
dtypes: int64(1), object(3)
memory usage: 63.2+ KB


,Serial Number,Error Type,Ungrammatical Statement,Standard English
1208,1209,Quantifier Errors,All of the students in this class are attentive.,All the students in this class are attentive.
1196,1197,Quantifier Errors,All of the paintings in this gallery are moder...,All the paintings in this gallery are modern art.
2008,2009,Lack of Parallelism in Lists or Series,The training program focuses on skill developm...,The training program focuses on skill developm...
473,474,Sentence Structure Errors,They visited their grandparents on the weekend.,They visited their grandparents on the weekend.
753,754,Run-on Sentences,He saw the advertisement he decided to apply.,"He saw the advertisement, and he decided to ap..."


In [23]:
df = df.rename(columns={'Ungrammatical Statement': 'incorrect', 'Standard English': 'correct'}) 
df.head()

,Serial Number,Error Type,incorrect,correct
0,1,Verb Tense Errors,I goes to the store everyday.,I go to the store everyday.
1,2,Verb Tense Errors,They was playing soccer last night.,They were playing soccer last night.
2,3,Verb Tense Errors,She have completed her homework.,She has completed her homework.
3,4,Verb Tense Errors,He don't know the answer.,He doesn't know the answer.
4,5,Verb Tense Errors,The sun rise in the east.,The sun rises in the east.


In [24]:
df.columns = [col.strip().lower() for col in df.columns]
df.sample(6)

,serial number,error type,incorrect,correct
1362,1363,Ambiguity,The car was drove by him.,The car was driven by him.
876,877,Mixed Metaphors/Idioms,39. You can catch more flies with honey than w...,39. You can catch more flies with honey than w...
406,407,Sentence Structure Errors,The childrens had a lot of fun at the playground.,The children had a lot of fun at the playground.
802,803,Sentence Fragments,"The computer, slow, needs an upgrade.",The computer is slow and needs an upgrade.
1,2,Verb Tense Errors,They was playing soccer last night.,They were playing soccer last night.
1819,1820,Relative Clause Errors,The cake I ate it for dessert was delicious.,The cake I ate for dessert was delicious.


In [25]:
df.isnull().sum()

serial number    0
error type       0
incorrect        0
correct          0
dtype: int64

In [26]:
df.duplicated().sum()

np.int64(0)

In [27]:
def clean_text(text):
    text = str(text).strip()
    text = " ".join(text.split())  # remove multiple spaces
    return text.lower()  # optional: only if you want case insensitivity

df["incorrect"] = df["incorrect"].apply(clean_text)
df["correct"] = df["correct"].apply(clean_text)
df.head()

,serial number,error type,incorrect,correct
0,1,Verb Tense Errors,i goes to the store everyday.,i go to the store everyday.
1,2,Verb Tense Errors,they was playing soccer last night.,they were playing soccer last night.
2,3,Verb Tense Errors,she have completed her homework.,she has completed her homework.
3,4,Verb Tense Errors,he don't know the answer.,he doesn't know the answer.
4,5,Verb Tense Errors,the sun rise in the east.,the sun rises in the east.


In [28]:
train_data = [
    {"input": "grammar: " + row["incorrect"], "output": row["correct"]}
    for _, row in df.iterrows()
]


In [29]:
pip install datasets


  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
Note: you may need to restart the kernel to use updated packages.


In [30]:
from datasets import Dataset

dataset = Dataset.from_list(train_data)
dataset = dataset.train_test_split(test_size=0.1)

dataset



DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 1816
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 202
    })
})

In [31]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [32]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"

tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)




In [33]:
pip install --upgrade torch huggingface_hub


Note: you may need to restart the kernel to use updated packages.


In [34]:
def preprocess(batch):
    input_enc = tokenizer(batch['input'], padding="max_length", truncation=True, max_length=128)
    target_enc = tokenizer(batch['output'], padding="max_length", truncation=True, max_length=128)
    
    input_enc['labels'] = target_enc['input_ids']
    return input_enc

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/1816 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]

In [35]:
pip install --upgrade --force-reinstall transformers


  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.30.1-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached typing_extensions-4.13.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached charset_normalizer

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", 
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
    push_to_hub=False,
)


In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


In [43]:
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.272000,0.230476
2,0.192100,0.177116
3,0.189200,0.166850


TrainOutput(global_step=681, training_loss=0.7794576070425969, metrics={'train_runtime': 3673.0035, 'train_samples_per_second': 1.483, 'train_steps_per_second': 0.185, 'total_flos': 184335533604864.0, 'train_loss': 0.7794576070425969, 'epoch': 3.0})

In [44]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.16685020923614502, 'eval_runtime': 31.5487, 'eval_samples_per_second': 6.403, 'eval_steps_per_second': 0.824, 'epoch': 3.0}


In [ ]:
model.save_pretrained("./grammar_model")
tokenizer.save_pretrained("./grammar_model")


In [46]:
pip install SpeechRecognition pyaudio



In [57]:
pip install transformers torch speechrecognition language-tool-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Manually set JAVA path for language_tool_python
os.environ["JAVA_HOME"] = r"C:\Users\priya\AppData\Local\Programs\Eclipse Adoptium\jdk-21.0.6.7-hotspot"
os.environ["PATH"] += os.pathsep + r"C:\Users\priya\AppData\Local\Programs\Eclipse Adoptium\jdk-21.0.6.7-hotspot\bin"


In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import speech_recognition as sr
import language_tool_python

# Load T5 model for grammar correction
model_path = "./grammar_model"  # or use "prithivida/grammar_error_correcter_v1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Initialize LanguageTool
tool = language_tool_python.LanguageTool('en-US')

# Initialize recognizer
r = sr.Recognizer()

# Record voice
with sr.Microphone() as source:
    print("🎤 Speak something...")
    audio = r.listen(source)

try:
    # Step 1: Transcribe voice
    text = r.recognize_google(audio)
    print("🗣 You said:", text)

    # Step 2: Check grammar errors before correction
    errors_before = len(tool.check(text))

    # Step 3: Correct grammar using T5
    input_ids = tokenizer.encode("fix: " + text, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    corrected = corrected.replace("fix:", "").strip()

    print("✅ Corrected Text:", corrected)

    # Step 4: Check grammar errors after correction
    errors_after = len(tool.check(corrected))

    # Step 5: Score logic (updated)
    if errors_before == 0 and errors_after == 0:
        score = "10/10 (Perfect)"
    elif errors_before == 0 and errors_after > 0:
        score = "1/10 (Correction made grammar worse)"
    elif errors_before > 0:
        improvement = errors_before - errors_after
        if improvement > 0:
            percent = int((improvement / errors_before) * 10)
            percent = min(10, max(1, percent))  # Clamp to 1–10
            score = f"{percent}/10 (Improved)"
        else:
            score = "1/10 (No improvement)"
    else:
        score = "1/10 (Unclear grammar state)"

    print("📊 Grammar Score:", score)

except sr.UnknownValueError:
    print("❌ Could not understand audio")
except sr.RequestError as e:
    print(f"⚠️ Error with Speech Recognition: {e}")


🎤 Speak something...
🗣 You said: I live in Jamshedpur
✅ Corrected Text: I live in Jamshedpur.
📊 Grammar Score: 10/10 (Perfect)
